## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,8.00,76,40,14.92,scattered clouds
1,1,Hobart,AU,-42.8794,147.3294,17.00,51,40,5.14,scattered clouds
2,2,Tual,ID,-5.6667,132.7500,28.17,72,99,2.31,overcast clouds
3,3,Yaremche,UA,48.4583,24.5519,11.00,32,75,2.00,broken clouds
4,4,Kieta,PG,-6.2167,155.6333,26.33,86,95,1.63,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 17
What is the maximum temperature you would like for your trip? 25


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
prefered_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 1 to 695
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              150 non-null    int64  
 1   City                 150 non-null    object 
 2   Country              150 non-null    object 
 3   Lat                  150 non-null    float64
 4   Lng                  150 non-null    float64
 5   Max Temp             150 non-null    float64
 6   Humidity             150 non-null    int64  
 7   Cloudiness           150 non-null    int64  
 8   Wind Speed           150 non-null    float64
 9   Current Description  150 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 12.9+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
prefered_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hobart,AU,-42.8794,147.3294,17.00,51,40,5.14,scattered clouds
9,9,Busselton,AU,-33.6500,115.3333,19.44,75,100,1.34,light rain
12,12,Hermanus,ZA,-34.4187,19.2345,23.33,52,5,1.79,clear sky
15,15,Port Alfred,ZA,-33.5906,26.8910,20.24,64,21,8.46,few clouds
23,23,Castro,BR,-24.7911,-50.0119,21.16,61,63,2.06,broken clouds
...,...,...,...,...,...,...,...,...,...,...
671,671,Kiunga,PG,-6.1219,141.2906,22.94,98,100,1.96,light rain
677,677,Yeppoon,AU,-23.1333,150.7333,23.00,78,90,3.09,overcast clouds
690,690,Dingle,PH,10.9995,122.6711,24.28,83,96,2.79,overcast clouds
694,694,La Baneza,ES,42.3003,-5.8977,22.78,25,20,5.66,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = prefered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hobart,AU,17.00,scattered clouds,-42.8794,147.3294,
9,Busselton,AU,19.44,light rain,-33.6500,115.3333,
12,Hermanus,ZA,23.33,clear sky,-34.4187,19.2345,
15,Port Alfred,ZA,20.24,few clouds,-33.5906,26.8910,
23,Castro,BR,21.16,broken clouds,-24.7911,-50.0119,
25,Bengkulu,ID,24.81,overcast clouds,-3.8004,102.2655,
26,Faya,SA,25.00,few clouds,18.3851,42.4509,
30,Kapaa,US,23.00,scattered clouds,22.0752,-159.3190,
35,Iringa,TZ,25.00,light rain,-7.7667,35.7000,
36,Alofi,NU,21.00,overcast clouds,-19.0595,-169.9187,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 1 to 695
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 150 non-null    object 
 1   Country              150 non-null    object 
 2   Max Temp             150 non-null    float64
 3   Current Description  150 non-null    object 
 4   Lat                  150 non-null    float64
 5   Lng                  150 non-null    float64
 6   Hotel Name           144 non-null    object 
dtypes: float64(3), object(4)
memory usage: 13.4+ KB


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °C</dd>
</dl>
"""
# <dt>Max Temp</dt><dd>{Max Temp} °C</dd>
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))